In [7]:
import csv
import cv2
import os
import traceback
import numpy as np
import tqdm

CSV_FILE="data/driving_log.csv"
IMG_DIR="data/IMG/"
STEERING_CORRECTION = 0.2

images = []
measurements = []


def read_csv(filename):
    with open(filename) as csvfile:
        rd = csv.DictReader(csvfile, fieldnames=["Center","Left","Right", "Steering", "Throttle", "Brake", "Speed"])
        for row in rd:
            center_image = cv2.imread(IMG_DIR + os.path.basename(row["Center"]))
            left_image = cv2.imread(IMG_DIR + os.path.basename(row["Left"]))
            right_image = cv2.imread(IMG_DIR + os.path.basename(row["Right"]))

            center_steering = float(row["Steering"])
            left_steering = center_steering + STEERING_CORRECTION
            right_steering = center_steering - STEERING_CORRECTION
            
            flipped_center = np.fliplr(center_image)
            flipped_steering = -center_steering
            
            images.extend([center_image, left_image, right_image, flipped_center])
            measurements.extend([center_steering, left_steering, right_steering, flipped_steering])
            

def read_csv_list(filename):
    with open(filename) as csvfile:
        rd = csv.reader(csvfile)
        for row in rd:
            center_image = cv2.imread(IMG_DIR + os.path.basename(row[0]))
            left_image = cv2.imread(IMG_DIR + os.path.basename(row[1]))
            right_image = cv2.imread(IMG_DIR + os.path.basename(row[2]))

            center_steering = float(row[3])
            left_steering = center_steering + STEERING_CORRECTION
            right_steering = center_steering - STEERING_CORRECTION

            flipped_center = np.fliplr(center_image)
            flipped_steering = -center_steering
            
            images.extend([center_image, left_image, right_image, flipped_center])
            measurements.extend([center_steering, left_steering, right_steering, flipped_steering])

In [8]:
%%time

print("Collecting dataset...."),
try:
    read_csv(CSV_FILE)
    X_train = np.array(images)
    Y_train = np.array(measurements)
except:
    traceback.print_exc()
print("done")
print(X_train.shape)
print(X_train.shape)

done
(72500, 160, 320, 3)
(72500, 160, 320, 3)
CPU times: user 1min 58s, sys: 26.1 s, total: 2min 24s
Wall time: 3min 8s


In [9]:
from keras.models import Sequential, Model
from keras.layers import Lambda, Flatten, Dense, Dropout
from keras.layers import Cropping2D
# from keras.layers.convolutional import Conv2D
# from keras.layers.pooling import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

print("Training model...")
model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
#cropping default 50,20
model.add(Cropping2D(cropping=((70,25), (0,0)), input_shape=(160,320,3)))
model.add(Conv2D(24, (5, 5), activation="relu", strides=(2, 2)))
model.add(Conv2D(36, (5, 5), activation="relu", strides=(2, 2)))
model.add(Conv2D(48, (5, 5), activation="relu", strides=(2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(Flatten()) #Flatten(input_shape=(160,320,3)
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))

model.compile(loss="mse", optimizer="adam")
model.fit(X_train, Y_train, validation_split=0.2, shuffle=True)

scores = model.evaluate(X_train, Y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Training model...


/Users/kolhea/anaconda3/envs/tensorflow_gpu/lib/python3.5/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  '` call to the Keras 2 API: ' + signature)
/Users/kolhea/anaconda3/envs/tensorflow_gpu/lib/python3.5/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  '` call to the Keras 2 API: ' + signature)
/Users/kolhea/anaconda3/envs/tensorflow_gpu/lib/python3.5/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
  '` call to the Keras 2 API: ' + signature)
/Users/kolhea/anaconda3/envs/tensorflow_gpu/lib/python3.5/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  '` call

ValueError: Error when checking model target: expected dense_7 to have shape (None, 10) but got array with shape (72500, 1)